In [557]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [558]:
# Importing the important librairies
import numpy as np
import pandas as pd
from math import sqrt, radians, cos, sin, asin
import datetime
from sklearn.linear_model import LinearRegression

In [559]:
# Loading the dataset
data = pd.read_csv('/content/drive/My Drive/Data_Science/Arrest_Data_from_2010_to_Present.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [560]:
data.shape

(1227521, 17)

In [561]:
data.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,4248313,02/24/2015,1310.0,20.0,Olympic,2022.0,37.0,M,H,5.0,Burglary,F,459PC,BURGLARY,5TH,WILTON,"(34.0653, -118.314)"
1,191811472,05/03/2019,1700.0,18.0,Southeast,1802.0,23.0,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)"
2,4254777,02/26/2015,2010.0,19.0,Mission,1985.0,22.0,M,H,6.0,Larceny,M,459.5PC,SHOPLIFTING,8300 VAN NUYS BL,NaN,"(34.2216, -118.4488)"
3,5614161,04/29/2019,1040.0,8.0,West LA,842.0,41.0,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)"
4,5615197,04/30/2019,615.0,6.0,Hollywood,663.0,27.0,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)"


In [562]:
data.shape

(1227521, 17)

In [563]:
data.isnull().sum()

Report ID                        0
Arrest Date                      4
Time                           199
Area ID                          4
Area Name                        4
Reporting District               4
Age                              4
Sex Code                         4
Descent Code                     4
Charge Group Code            87570
Charge Group Description     88006
Arrest Type Code                 5
Charge                           5
Charge Description           87534
Address                          5
Cross Street                520187
Location                         5
dtype: int64

In [564]:
data.dtypes

Report ID                    object
Arrest Date                  object
Time                        float64
Area ID                     float64
Area Name                    object
Reporting District          float64
Age                         float64
Sex Code                     object
Descent Code                 object
Charge Group Code           float64
Charge Group Description     object
Arrest Type Code             object
Charge                       object
Charge Description           object
Address                      object
Cross Street                 object
Location                     object
dtype: object

#**How many bookings of arrestees were made in 2018 ?**

In [565]:
# Retrieves the year from the date column
data['year'] = pd.DatetimeIndex(data['Arrest Date']).year

In [566]:
# Counts the number of bookings of arrestees per year by counting the area id
booking_of_arestees = data.groupby(['year'])['Area ID'].count().reset_index()

In [567]:
print('The number of booking of arrestees in 2018 is ' + str(booking_of_arestees['Area ID'][booking_of_arestees['year'] == 2018.0].tolist()[0]))

The number of booking of arrestees in 2018 is 42381


#**How many bookings of arrestees were made in the area with the most arrests in 2018?**

In [568]:
# Selects the dataset only containing information for 2018
dataset_2018 = data[data['year'] == 2018.0]

In [569]:
dataset_2018['Arrest Type Code'].unique()

array(['F', 'M', 'I', 'O', 'D', nan], dtype=object)

In [570]:
dataset_2018.head(2)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year
165,5213460,01/24/2018,1930.0,6.0,Hollywood,668.0,29.0,M,H,10.0,Fraud/Embezzlement,F,530.5(A)PC,GET CREDIT/ETC OTHER'S ID,ST ANDREWS,FOUNTAIN,"(34.0949, -118.3109)",2018.0
166,5257944,03/15/2018,310.0,7.0,Wilshire,743.0,33.0,F,O,1.0,Homicide,F,191.5(B)PC,VEH MANSLAUGHTR/OPER VESSEL DUI & DUR FEL,OLYMPIC,MASSELIN,"(34.0574, -118.3547)",2018.0


In [571]:
# Groups by Area name to get the area with the highest count
data_area = dataset_2018.groupby(['Area Name'])['Area ID'].count().sort_values().reset_index()
data_area

,Area Name,Area ID
0,Wilshire,615
1,Southeast,764
2,Foothill,801
3,West Valley,955
4,Hollenbeck,995
5,Topanga,1208
6,Devonshire,1336
7,Olympic,1350
8,West LA,1415
9,77th Street,1457


In [572]:
print('The highest number of arrests is '+str(data_area['Area ID'][data_area['Area Name'] == 'Central'].tolist()[0])+' and was made in the area called Central')

The highest number of arrests is 5621 and was made in the area called Central


# **Finding the average age of arrestees per each charge group**

In [573]:
# Calculates the age average by average group
average_list = dataset_2018.groupby(['Charge Group Description'])['Age'].mean().reset_index()

In [574]:
# Dropps the two columns that are not needed
average_list = average_list[average_list['Charge Group Description']!= 'Pre-Delinquency']
average_list = average_list[average_list['Charge Group Description'] != 'Non-Criminal Detention']

In [575]:
# Below is list of the averages of the ages by Charge Group
average_list

,Charge Group Description,Age
0,Against Family/Child,36.950355
1,Aggravated Assault,33.907359
2,Burglary,30.366755
3,Disorderly Conduct,41.924290
4,Disturbing the Peace,39.727273
5,Driving Under Influence,34.428291
6,Drunkeness,44.219972
7,Federal Offenses,34.600000
8,Forgery/Counterfeit,35.066667
9,Fraud/Embezzlement,35.689076


# **95% quantile of the age of the arrestee**

In [576]:
# Selecting the dataframes using the conditions
array = ['Vehicle Theft','Robbery','Burglary','Receive Stolen Property']
df_for_quantile = dataset_2018[dataset_2018['Charge Group Description'].isin(array)]
df_for_quantile.head(2)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year
178,5449454,10/15/2018,2330.0,1.0,Central,157.0,43.0,M,H,7.0,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,SPRING,5TH,"(34.0416, -118.244)",2018.0
179,5449463,10/15/2018,2330.0,1.0,Central,154.0,32.0,M,H,7.0,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,SPRING ST,6TH,"(34.046, -118.2509)",2018.0


In [577]:
# Calculating the 95% quantile
quantile = df_for_quantile['Age'].quantile(.95)

In [578]:
print("The quantile for the is " + str(quantile))

The quantile for the is 52.0


# **Felony arrest incidents have been dropping over the years**

In [579]:
data.head(2)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year
0,4248313,02/24/2015,1310.0,20.0,Olympic,2022.0,37.0,M,H,5.0,Burglary,F,459PC,BURGLARY,5TH,WILTON,"(34.0653, -118.314)",2015.0
1,191811472,05/03/2019,1700.0,18.0,Southeast,1802.0,23.0,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)",2019.0


In [580]:
# We will filter the dataset only taking felony crimes represented by Arrest Type Code F
Felony_data_set = data[data['Arrest Type Code'] == 'F']

Felony_list = Felony_data_set.groupby(['year'])['Area ID'].count().reset_index()
Felony_list

,year,Area ID
0,2010.0,51155
1,2011.0,49830
2,2012.0,50610
3,2013.0,47880
4,2014.0,45878
5,2015.0,35709
6,2016.0,36880
7,2017.0,35973
8,2018.0,6308
9,2019.0,19425


In [581]:
X = np.array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])
X = np.expand_dims(X, axis=1)
y = np.array([51155, 49830, 50610, 47880, 45878, 35709, 36880, 35973, 6308])

model = LinearRegression()
model.fit(X, y)
X_test = [2019]
y_pred = model.predict([X_test])
print("The estimated value number of felony for 2019 is "+str(y_pred))

The estimated value number of felony for 2019 is [18308.94444445]


# **How many arrest incidents occurred within 2 km from the Bradbury Building in 2018?**

In [582]:
# Distance between two points
def findDistance(latitude1, longitude1, latitude2, longitude2):
    radius = 6371
  
    # Converting degrees into radians
    latitude1 = radians(latitude1)
    latitude2 = radians(latitude2)
    longitude1 = radians(longitude1)
    longitude2 = radians(longitude2)
    
    longdiff = longitude2 - longitude1
    latdiff = latitude2 - latitude1
    
    f = sin(latdiff / 2)**2 + cos(latitude1) * cos(latitude2) * sin(longdiff / 2)**2
    
    c = asin(sqrt(f)) * 2
    
    return radius*c

In [583]:
# Generating the total distance between the points
distance = []
location = data['Location']
for i in range(len(location)):
    row = location[i]
    if (type(row) == str):
        text = row[1:-1]
        array = text.split(",")
        latitude = float(array[0])
        longitude = float(array[1])
        dis = findDistance(34.050536, -118.247861, latitude, longitude)
        distance.append(dis)
    else:
        distance.append(np.nan)

In [584]:
data.insert(17, "distance", distance, True)
data.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,distance,year
0,4248313,02/24/2015,1310.0,20.0,Olympic,2022.0,37.0,M,H,5.0,Burglary,F,459PC,BURGLARY,5TH,WILTON,"(34.0653, -118.314)",6.310144,2015.0
1,191811472,05/03/2019,1700.0,18.0,Southeast,1802.0,23.0,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)",11.172483,2019.0
2,4254777,02/26/2015,2010.0,19.0,Mission,1985.0,22.0,M,H,6.0,Larceny,M,459.5PC,SHOPLIFTING,8300 VAN NUYS BL,NaN,"(34.2216, -118.4488)",26.529891,2015.0
3,5614161,04/29/2019,1040.0,8.0,West LA,842.0,41.0,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)",19.470628,2019.0
4,5615197,04/30/2019,615.0,6.0,Hollywood,663.0,27.0,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)",9.459945,2019.0


In [585]:
# Filtering only for 2018
distance_df = data[data['year'] == 2018.0]

# Filtering for Location different from (0,0)
distance_df = distance_df[distance_df['Location'] != '(0,0)']
print("The number of arrests that occurred within 2 km from the Bradbury Building in 2018 is "+str(len(distance_df[distance_df['distance'] <= 2])))

The number of arrests that occurred within 2 km from the Bradbury Building in 2018 is 5913


# **How many arrest incidents were made per kilometer on Pico Boulevard during 2018?**

In [586]:
# We will start with the 2018 dataset
dataset_2018.head(2)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year
165,5213460,01/24/2018,1930.0,6.0,Hollywood,668.0,29.0,M,H,10.0,Fraud/Embezzlement,F,530.5(A)PC,GET CREDIT/ETC OTHER'S ID,ST ANDREWS,FOUNTAIN,"(34.0949, -118.3109)",2018.0
166,5257944,03/15/2018,310.0,7.0,Wilshire,743.0,33.0,F,O,1.0,Homicide,F,191.5(B)PC,VEH MANSLAUGHTR/OPER VESSEL DUI & DUR FEL,OLYMPIC,MASSELIN,"(34.0574, -118.3547)",2018.0


#### ***Step 1 : Select the dataframe where the listed address mentions "PICO"***






In [587]:
# Selects the dataframe with no Na values
dataset_2018 = dataset_2018[pd.notnull(dataset_2018['Address'])]

# Selects the dataframe where the listed address mentions "Pico"
data_frame_with_pico = dataset_2018[dataset_2018['Address'].str.contains("PICO")]
data_frame_with_pico.shape

(269, 18)

In [588]:
data_frame_with_pico.head(2)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year
193,5485873,12/01/2018,1350.0,20.0,Olympic,2076.0,37.0,M,B,8.0,Other Assaults,M,243(B)PC,BATTERY ON PO/EMERGENCY PERSONNEL W/O INJ,PICO BL,BERENDO ST,"(34.0473, -118.2936)",2018.0
4738,5270090,03/29/2018,931.0,8.0,West LA,836.0,53.0,M,W,6.0,Larceny,F,487(A)PC,GRAND THEFT MONEY/PROPERTY > $400,PICO BL,PROSSER ST,"(34.0524, -118.4248)",2018.0


In [589]:
# Splitting the columns into Location columns into Latitude and Longitude
pico_data = data_frame_with_pico['Location'].str[1:-1].str.split(',',expand = True)
pico_data.columns = ['Latitude', 'Longitude']
dataset_2018['Latitude'] = pico_data['Latitude']
dataset_2018['Longitude'] = pico_data['Longitude']

#### ***Step 2 : Remove outliers by filtering out locations where either the latitude or longitude is 2 standard deviations beyond the mean***

In [590]:
# identify outliers with standard deviation
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
Longitude = pd.to_numeric(dataset_2018['Longitude'])
Latitude = pd.to_numeric(dataset_2018['Latitude'])

# Removing values for Longitude outliers
data_mean, data_std = mean(Longitude), std(Longitude)
# identify outliers
cut_off = data_std * 2
lower, upper = data_mean - cut_off, data_mean + cut_off
# remove outliers
no_outliers = dataset_2018[Longitude >= lower] 
no_outliers = no_outliers[Longitude <= upper]

# Removing values for Latitude outliers
data_mean1, data_std1 = mean(Latitude), std(Latitude)
# identify outliers
cut_off1 = data_std1 * 2
lower1, upper1 = data_mean1 - cut_off1, data_mean1 + cut_off1
# remove outliers
no_outliers = no_outliers[Latitude >= lower1] 
no_outliers = no_outliers[Latitude <= upper1]
no_outliers.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,year,Latitude,Longitude
193,5485873,12/01/2018,1350.0,20.0,Olympic,2076.0,37.0,M,B,8.0,Other Assaults,M,243(B)PC,BATTERY ON PO/EMERGENCY PERSONNEL W/O INJ,PICO BL,BERENDO ST,"(34.0473, -118.2936)",2018.0,34.0473,-118.2936
4738,5270090,03/29/2018,931.0,8.0,West LA,836.0,53.0,M,W,6.0,Larceny,F,487(A)PC,GRAND THEFT MONEY/PROPERTY > $400,PICO BL,PROSSER ST,"(34.0524, -118.4248)",2018.0,34.0524,-118.4248


In [591]:
# Let's now find the most Eastern and the most Western points

# Converting the two columns to floats
no_outliers['Longitude'] = no_outliers['Longitude'].astype(float)
no_outliers['Latitude'] = no_outliers['Latitude'].astype(float)

#### ***Step 3 : Calculate the distance from the most western and eastern coordinate points.***

In [592]:
# The most Eastern point is the point with the biggest longitute, 
# while the point with the Least longitude is the most western point

min_longitude = no_outliers['Longitude'].min()
max_longitude = no_outliers['Longitude'].max()
print("The minimum longitude is "+str(min_longitude)+ " The longitude latitude is " +str(max_longitude))

The minimum longitude is -118.453 The longitude latitude is -118.2484


In [593]:
# Calculating the Latitude of the points
min_long_latitude = no_outliers['Latitude'][no_outliers['Longitude'] == min_longitude]
max_long_latitude = no_outliers['Latitude'][no_outliers['Longitude'] == max_longitude]
print("The latitude of the minimum longitude is "+str(min_long_latitude.tolist()[0])+ " The latitude of the maximum longitude is " +str(max_long_latitude.tolist()[0]))

The latitude of the minimum longitude is 34.0281 The latitude of the maximum longitude is 34.0303


In [594]:
# Now that we know we most Eastern and the most Western point, we can caluculate the distance between them
# For that we will use the function findDistance, that we use above
length_pico = findDistance(min_long_latitude.tolist()[0],min_longitude, max_long_latitude.tolist()[0], max_longitude)
print("From this estimations, the Pico Boulevard length is "+str(length_pico))

From this estimations, the Pico Boulevard length is 18.856102139122424


#### ***Step 4 : Calculate number of arrest per kilometer on the PICO boulevard***

In [596]:
# We have the length of the pico boulevard, we can now calculate the number of arrest by Kilometer 
# By dividing the number of arrest by the length of the Pico boulevard
number_of_arrests_pico = no_outliers.shape[0]
number_of_arrests_per_kilo = number_of_arrests_pico//length_pico
print("The number of arrests per kilometer on the PICO boulevard is "+str(number_of_arrests_per_kilo))

The number of arrests per kilometer on the PICO boulevard is 14.0
